# Anomaly detection ANN
That model based on an Artificial Neural Network (ANN) and using k-fold cross-validation for model fitting

## Imports

In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

## Data preparation

In [2]:
dataframe = pd.read_csv('ecg_dataset.csv', header=None)
raw_data = dataframe.values

labels = raw_data[:, -1]
data = raw_data[:, 0:-1]

train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=50)

train_data[0]

array([0.38258117, 0.34857435, 0.29507594, 0.2360118 , 0.2055155 ,
       0.22841321, 0.31389438, 0.34302886, 0.36067474, 0.36985195,
       0.40503097, 0.38827506, 0.39750014, 0.41477571, 0.42704026,
       0.43425207, 0.43815893, 0.44374103, 0.44351909, 0.44594805,
       0.45034206, 0.45298267, 0.45071413, 0.45236969, 0.44929805,
       0.45240395, 0.45156713, 0.45199604, 0.44638053, 0.44333211,
       0.44028045, 0.44643116, 0.44133146, 0.43481545, 0.43392774,
       0.43273256, 0.43288096, 0.43452733, 0.43671602, 0.43852113,
       0.44137622, 0.44369191, 0.45070433, 0.45913821, 0.45981626,
       0.46429308, 0.45783317, 0.46210159, 0.46010114, 0.46217812,
       0.46326499, 0.47080483, 0.4709795 , 0.46228708, 0.46262172,
       0.46095022, 0.45744584, 0.45504108, 0.4518767 , 0.46504835,
       0.46807957, 0.47459668, 0.47899268, 0.48987364, 0.48922653,
       0.49118877, 0.48572023, 0.49434313, 0.5118916 , 0.52040492,
       0.51698048, 0.52387298, 0.52351826, 0.52580741, 0.52217

## ANN model

In [3]:
def build_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(32, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5),
        loss=tf.keras.losses.mse,
        metrics=[tf.keras.metrics.binary_accuracy]
    )
    return model


## Model training
Below there is a process of fitting model using k-fold cross-validation

In [4]:
k = int(4)
num_val_samples = len(train_data) // k
epochs = 5
all_scores = []

for i in range(k):
    print(f'processing fold #{i}')
    val_data = train_data[i * num_val_samples: (i+1) * num_val_samples]
    val_labels = train_labels[i*num_val_samples: (i+1) * num_val_samples]

    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i+1) * num_val_samples:]],
        axis=0
    )

    partial_train_labels = np.concatenate(
        [train_labels[:i * num_val_samples],
        train_labels[(i+1) * num_val_samples:]],
        axis=0
    )

    model = build_model()

    model.fit(
        partial_train_data,
        partial_train_labels,
        epochs=epochs,
        batch_size=128
    )

    val_mse, val_mae = model.evaluate(val_data, val_labels, verbose=0)
    all_scores.append(val_mae)

processing fold #0
Epoch 1/5
75/75 [==============================] - 5s 59ms/step - loss: 0.1762 - binary_accuracy: 0.7486
Epoch 2/5
75/75 [==============================] - 5s 63ms/step - loss: 0.1661 - binary_accuracy: 0.7486
Epoch 3/5
75/75 [==============================] - 5s 64ms/step - loss: 0.1538 - binary_accuracy: 0.7486
Epoch 4/5
75/75 [==============================] - 4s 56ms/step - loss: 0.1359 - binary_accuracy: 0.7486
Epoch 5/5
75/75 [==============================] - 5s 63ms/step - loss: 0.1120 - binary_accuracy: 0.7842
processing fold #1
Epoch 1/5
75/75 [==============================] - 4s 56ms/step - loss: 0.1803 - binary_accuracy: 0.7448
Epoch 2/5
75/75 [==============================] - 4s 60ms/step - loss: 0.1641 - binary_accuracy: 0.7448
Epoch 3/5
75/75 [==============================] - 4s 56ms/step - loss: 0.1400 - binary_accuracy: 0.7448
Epoch 4/5
75/75 [==============================] - 5s 60ms/step - loss: 0.1060 - binary_accuracy: 0.8338
Epoch 5/5
75/75 [

## Model saving

In [27]:
model.save('models/detectors/ann')

INFO:tensorflow:Assets written to: models/detectors/ann\assets


INFO:tensorflow:Assets written to: models/detectors/ann\assets


## Model evaluating and testing

In [7]:
test_mse_score, test_mae_score = model.evaluate(test_data, test_labels)

100/100 [==============================] - 3s 26ms/step - loss: 0.0651 - binary_accuracy: 1.0000


In [24]:
test_sample = test_data[0:1]

In [26]:
x = model.predict(test_sample)

1/1 [==============================] - 0s 89ms/step
